# Setting up ESP8266 WiFi using MicroPython
In this notebook we will discuss how to prepare and use the ESP8266 WiFi. The python commands used in this notebook may not work on any machine but they are guaranteed to work on an ESP8266 microcontroller with MicroPython firmware.

**Contents:**
<I>
1. Importing the necessary packages
2. Setting up WiFi as access point (Server)
3. Setting up WiFi as station (Client)
4. Socket types
5. Creating a socket (Server)
6. Creating a socket (Client)
7. Define the response web page
8. Sending and receiving data
</I>

## Access point interface
### Importing necessary packages
In this section we import the packages needed to access the ESP8266 built-in WiFi
Required packages:
- usocket/socket
- network

In [ ]:
try:
    import usocket as socket  # try to import usocket
except:
    import socket             # if usocket was not found, import socket instead
import network                # import the network library
import gc                     # import garbage collector
gc.collect()

### Setting up WiFi as an access point (server.)
In this section we set up the ESP8266 by: First, starting WLAN acess point interface. Then, activating and configuring the credentials. Finally, we wait for the access point to be activated then proceed.

In [ ]:
# hold your network credentials
ssid = 'myNetworkName'
password = 'myPassword'

ap = network.WLAN(network.AP_IF)          # start WLAN access point interface
ap.active(True)                           # activate the access point
ap.config(essid=ssid, password=password)  # configure the SSID and Password

# wait for connection
while ap.active() == False:
    pass

### Setting up WiFi as a station (client.)
In this section we set up the ESP8266 by: First, starting WLAN station interface. Then, activating and connecting to the network. Finally, we wait for the connection to be activated then proceed.

In [ ]:
ssid = 'myNetworkName'
password = 'myPassword'

station = network.WLAN(network.STA_IF)  # start WLAN station interface
station.active(True)                    # activate the station
station.connect(ssid, password)         # connect through the SSID and Password

# wait for connection
while station.isconnected() == False:
    pass

### Socket types
![socket-server image](socket-server.webp)

### Creating a socket (Server.)

In [ ]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('', 80))  # bind the socket to an address
s.listen(5)       # enable the server to accept connections

### Creating a socket (Client.)


In [ ]:
address = 127.0.0.1

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((address, 80))  # enable the server to accept connections

### Define the response web page
Here we define a function that returns an HTML web page that will be sent in response to incoming data.

In [ ]:
def web_page(req):  
    html = """<html>
    <head>
        <meta name="viewport" content="width=device-width, initial-scale=1"><meta http-equiv="refresh" content="1">
    </head>
    <body>
        <p><strong>""" + req + """</strong></p>
    </body>
    </html>"""

    return html

### Sending and receiving data
In this section we will create a loop where the access point accepts a connection and when a client connects, the server accepts the connection, then, saves a new socket object to accept and send data on the `conn` variable, and saves the client address to connect to the server on the `addr` variable, then receive data and reply with the web page we defined above.

In [ ]:
# server
while True:
    conn, addr = s.accept()
    conn.settimeout(3.0)
    request = conn.recv(1024)
    conn.settimeout(None)
    request = str(request)
    
    response = web_page(request)
    conn.send('HTTP/1.1 200 OK\n')
    conn.send('Content-Type: text/html\n')
    conn.send('Connection: close\n\n')
    conn.sendall(response)
    
    conn.close()

In [ ]:
# client
while True:
    try:
        request = conn.recv(1024)
        request = str(request)

        response = web_page(request)
        conn.send('HTTP/1.1 200 OK\n')
        conn.send('Content-Type: text/html\n')
        conn.send('Connection: close\n\n')
        conn.sendall(response)
    except:
        address = 127.0.0.1
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect((address, 80))